In [ ]:
# Training Pipeline Notebook

In [1]:
# get dataset
from pathlib import Path

from datasets import load_dataset, Split

from data import *

data_dir = Path('resources/bart/')

data_files = {
    Split.TRAIN: str((data_dir / 'train.history_belief').absolute()),
    Split.VALIDATION: str((data_dir / 'val.history_belief').absolute()),
    Split.TEST: str((data_dir / 'test.history_belief').absolute())
}

dataset = load_dataset('data/dataset/multiwoz_dataset.py', data_files=data_files)
dataset = dataset.map(flatten_conversation, batched=True, remove_columns=dataset['train'].column_names)
masked_deltas = dataset.map(mask_delta_beliefs, remove_columns='turn')

Using custom data configuration default-bfc7feb1c1e93a63
Reusing dataset multi_woz_dataset (/Users/daohuei/.cache/huggingface/datasets/multi_woz_dataset/default-bfc7feb1c1e93a63/0.0.0/214b5f86ecf324bff982d075298ac2b6490bbca4895dda93d15c69bf8e8274e7)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /Users/daohuei/.cache/huggingface/datasets/multi_woz_dataset/default-bfc7feb1c1e93a63/0.0.0/214b5f86ecf324bff982d075298ac2b6490bbca4895dda93d15c69bf8e8274e7/cache-d16f23eccc4b23c9.arrow
Loading cached processed dataset at /Users/daohuei/.cache/huggingface/datasets/multi_woz_dataset/default-bfc7feb1c1e93a63/0.0.0/214b5f86ecf324bff982d075298ac2b6490bbca4895dda93d15c69bf8e8274e7/cache-729a1d081f6da48b.arrow
Loading cached processed dataset at /Users/daohuei/.cache/huggingface/datasets/multi_woz_dataset/default-bfc7feb1c1e93a63/0.0.0/214b5f86ecf324bff982d075298ac2b6490bbca4895dda93d15c69bf8e8274e7/cache-68c0f4701c954fae.arrow


  0%|          | 0/56778 [00:00<?, ?ex/s]

  0%|          | 0/7374 [00:00<?, ?ex/s]

  0%|          | 0/7372 [00:00<?, ?ex/s]

In [86]:
from datasets import Dataset
sample_dataset = Dataset.from_dict(masked_deltas["validation"][:2])
sample_dataset_2 = Dataset.from_dict(masked_deltas["validation"][50:55])

In [87]:
# TODO: add special tokens
from transformers import BartTokenizer
tok = BartTokenizer.from_pretrained("facebook/bart-base")

tok.add_special_tokens(
    {"additional_special_tokens": [
        "<|context|>", 
        "<|user|>", 
        "<|system|>", 
        "<|endofcontext|>", 
        "<|previousbelief|>", 
        "<|endofpreviousbelief|>", 
        "<|belief|>",
        "<|endofbelief|>",
    ]}
)


loading file https://huggingface.co/facebook/bart-base/resolve/main/vocab.json from cache at /Users/daohuei/.cache/huggingface/transformers/43978bdeaa326572886b44fcfed82f932f76571095ce31973e51c3da8ccade7f.d67d6b367eb24ab43b08ad55e014cf254076934f71d832bbab9ad35644a375ab
loading file https://huggingface.co/facebook/bart-base/resolve/main/merges.txt from cache at /Users/daohuei/.cache/huggingface/transformers/3c167ed8af56e6605eeb794b63a79d65d85e6708c9b04408d41946337030f5cd.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
loading file https://huggingface.co/facebook/bart-base/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/facebook/bart-base/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/facebook/bart-base/resolve/main/tokenizer_config.json from cache at None
loading configuration file https://huggingface.co/facebook/bart-base/resolve/main/config.json from cache at /Users/daohuei/.cache/huggi

8

In [88]:
len(tok)

50273

In [89]:
[len(x) for x in tokenization(sample_dataset_2)["labels"]]

[533, 533, 533, 533, 533]

In [90]:
# tokenization
def tokenization(examples):
    tokenized_examples = tok(examples["masked"], padding=True)
    tokenized_examples["labels"] = tok(examples["target"], padding=True)["input_ids"]
    return tokenized_examples
train_set = sample_dataset.map(tokenization, batched=True,
            remove_columns=sample_dataset.column_names)
            #, remove_columns='turn')
train_set_2 = sample_dataset_2.map(tokenization, batched=True,
            remove_columns=sample_dataset_2.column_names)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [91]:
# get the model
from transformers import BartForConditionalGeneration, BartConfig

model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")

model.resize_token_embeddings(len(tok))


example_english_phrase = "UN Chief Says There Is No <mask> in Syria"
batch = tok(example_english_phrase, return_tensors="pt")
generated_ids = model.generate(batch["input_ids"])
tok.batch_decode(generated_ids, skip_special_tokens=True)

loading configuration file https://huggingface.co/facebook/bart-base/resolve/main/config.json from cache at /Users/daohuei/.cache/huggingface/transformers/f5310d276a6d1648d00c32fadc8bf7b4607e0fbd5b404fc4a0045960aa2bdfdb.a243ed957122436adb0b8d8e9d20f896f45c174b6324d625ca0a20a84f72a910
Model config BartConfig {
  "_name_or_path": "bart-base",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartModel"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id":

['UN Chief Says There Is No War in Syria']

In [92]:
# define the trainer

In [93]:
# customize the training loop => for different trainloader

In [105]:
from transformers import (
    AutoModelForQuestionAnswering,
    TrainingArguments,
    Trainer,
)


In [106]:
from transformers import TrainerCallback
from transformers import default_data_collator

# class MyCallback(TrainerCallback):
#     "A callback that prints a message at the beginning of training"

#     def on_train_begin(self, args, state, control, **kwargs):
#         print("Starting training")
        
#     def on_epoch_begin(self, args, state, control, **kwargs):
#         print(state)
#         print(control)
#         print(kwargs)

data_collator = default_data_collator

In [109]:
def test_compute_metrics(eval_predictions):
    logits, hidden_values = eval_predictions.predictions
    print(tok.batch_decode(logits.argmax(-1)))
    return {"score": 100}

CURRICULUM_DATASET = [train_set, train_set_2]
class CurriculumTrainer(Trainer):
    def __init__(self, curriculum_dataset, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.curriculum_dataset = curriculum_dataset
        
    def curriculum_train(self):
        
        self.total_epochs = self.args.num_train_epochs
        self.final_train_dataset = self.train_dataset
        self.model_name = self.args.output_dir
        
        for idx, c_dataset in enumerate(self.curriculum_dataset):
            # only train for 1 epoch during curriculum training
            self.args.num_train_epochs = 1
            self.args.output_dir = f"{self.model_name}/course_{idx}"
            self.train_dataset = c_dataset
            self.train()
            
        # switch back to the final training on the most difficult 
        self.args.num_train_epochs = self.total_epochs
        self.args.output_dir = f"{self.model_name}/final"
        self.train_dataset = self.final_train_dataset
        self.train()
#         self.train_dataset = CURRICULUM_DATASET[1]
#         self.train()
        
    # def get_train_dataloader():
    #     pass
    # def compute_loss(self, model, inputs):
    #     labels = inputs.pop("labels")
    #     outputs = model(**inputs)
    #     logits = outputs[0]
    #     return my_custom_loss(logits, labels)
    
    

name = "test_trainer"
BATCH_SIZE = 2
EPOCHS = 3
# setup trainer
args = TrainingArguments(
    output_dir=f"checkpoints/{name}",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    weight_decay=0.01,
    dataloader_num_workers=0,
    local_rank=-1,
    load_best_model_at_end=True,
    # resume_from_checkpoint=f"{name}/checkpoint-19000",
)
data_collator = default_data_collator
trainer = CurriculumTrainer(
    CURRICULUM_DATASET,
    model,
    args,
    train_dataset=train_set,
    eval_dataset=train_set,
    data_collator=data_collator,
    compute_metrics=test_compute_metrics
    # callbacks=[MyCallback],  # We can either pass the callback class this way or an instance of it (MyCallback())
)
trainer.curriculum_train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/Users/daohuei/miniconda3/envs/py3_8/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2
  Num Epochs = 1
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 1


Epoch,Training Loss,Validation Loss,Score
1,No log,0.739805,100


***** Running Evaluation *****
  Num examples = 2
  Batch size = 2


['<s>-.ii need to book a hotel in the east that has 4 stars. New Iattraction area not mentioned, attraction name not mentioned, attraction type not mentioned, hospital department not mentioned, hotel area not mentioned, hotel book day not mentioned, hotel book people not mentioned, hotel book stay not mentioned, hotel internet not mentioned, hotel name not mentioned, hotel parking not mentioned, hotel pricerange not mentioned, hotel stars not mentioned, hotel type not mentioned, restaurant area not mentioned, restaurant book day not mentioned, restaurant book people not mentioned, restaurant book time not mentioned, restaurant food not mentioned, restaurant name not mentioned, restaurant pricerange not mentioned, taxi arriveby not mentioned, taxi departure not mentioned, taxi destination not mentioned, taxi leaveat not mentioned, train arriveby not mentioned, train book people not mentioned, train day not mentioned, train departure not mentioned, train destination not mentioned, train 

Saving model checkpoint to checkpoints/test_trainer/course_0/checkpoint-1
Configuration saved in checkpoints/test_trainer/course_0/checkpoint-1/config.json
Model weights saved in checkpoints/test_trainer/course_0/checkpoint-1/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from checkpoints/test_trainer/course_0/checkpoint-1 (score: 0.7398045063018799).
***** Running training *****
  Num examples = 5
  Num Epochs = 1
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 3


...

Saving model checkpoint to checkpoints/test_trainer/final/checkpoint-1
Configuration saved in checkpoints/test_trainer/final/checkpoint-1/config.json
Model weights saved in checkpoints/test_trainer/final/checkpoint-1/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 2
  Batch size = 2


['<s>-.ii need to book a hotel in the east that has 4 stars. New Iattraction area not mentioned, attraction name not mentioned, attraction type not mentioned, hospital department not mentioned, hotel area not mentioned, hotel book day not mentioned, hotel book people not mentioned, hotel book stay not mentioned, hotel internet not mentioned, hotel name not mentioned, hotel parking not mentioned, hotel pricerange not mentioned, hotel stars not mentioned, hotel type not mentioned, restaurant area not mentioned, restaurant book day not mentioned, restaurant book people not mentioned, restaurant book time not mentioned, restaurant food not mentioned, restaurant name not mentioned, restaurant pricerange not mentioned, taxi arriveby not mentioned, taxi departure not mentioned, taxi destination not mentioned, taxi leaveat not mentioned, train arriveby not mentioned, train book people not mentioned, train day not mentioned, train departure not mentioned, train destination not mentioned, train 

Saving model checkpoint to checkpoints/test_trainer/final/checkpoint-2
Configuration saved in checkpoints/test_trainer/final/checkpoint-2/config.json
Model weights saved in checkpoints/test_trainer/final/checkpoint-2/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 2
  Batch size = 2


['<s>-.ii need to book a hotel in the east that has 4 stars. New Iattraction area not mentioned, attraction name not mentioned, attraction type not mentioned, hospital department not mentioned, hotel area not mentioned, hotel book day not mentioned, hotel book people not mentioned, hotel book stay not mentioned, hotel internet not mentioned, hotel name not mentioned, hotel parking not mentioned, hotel pricerange not mentioned, hotel stars not mentioned, hotel type not mentioned, restaurant area not mentioned, restaurant book day not mentioned, restaurant book people not mentioned, restaurant book time not mentioned, restaurant food not mentioned, restaurant name not mentioned, restaurant pricerange not mentioned, taxi arriveby not mentioned, taxi departure not mentioned, taxi destination not mentioned, taxi leaveat not mentioned, train arriveby not mentioned, train book people not mentioned, train day not mentioned, train departure not mentioned, train destination not mentioned, train 

Saving model checkpoint to checkpoints/test_trainer/final/checkpoint-3
Configuration saved in checkpoints/test_trainer/final/checkpoint-3/config.json
Model weights saved in checkpoints/test_trainer/final/checkpoint-3/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from checkpoints/test_trainer/final/checkpoint-1 (score: 0.7398045063018799).


In [ ]:


#     predictions (np.ndarray) — Predictions of the model.
# label_ids (np.ndarray) — Targets to be matched.
# inputs (np.ndarray, optional) —
    